In [1]:
import tensorflow as tf
import os

In [ ]:
dataset_url ="dataset/PetImages"
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join(dataset_url, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            img_bytes = tf.io.read_file(fpath)
            tf.io.decode_image(img_bytes)
        except Exception as e:
            print(f"Bad file found, deleting: {fpath}")
            num_skipped += 1
            os.remove(fpath)

Bad file found, deleting: dataset/PetImages\Cat\10404.jpg
Bad file found, deleting: dataset/PetImages\Cat\4351.jpg
Bad file found, deleting: dataset/PetImages\Cat\666.jpg
Bad file found, deleting: dataset/PetImages\Cat\Thumbs.db
Bad file found, deleting: dataset/PetImages\Dog\11233.jpg
Bad file found, deleting: dataset/PetImages\Dog\11702.jpg
Bad file found, deleting: dataset/PetImages\Dog\11912.jpg
Bad file found, deleting: dataset/PetImages\Dog\2317.jpg
Bad file found, deleting: dataset/PetImages\Dog\2494.jpg
Bad file found, deleting: dataset/PetImages\Dog\9500.jpg
Bad file found, deleting: dataset/PetImages\Dog\Thumbs.db


In [2]:
image_size=(256,256)
batch_size = 32

In [8]:
train_dr = tf.keras.utils.image_dataset_from_directory(
    dataset_url,
    subset="training",
    seed=123,
    validation_split=0.2,
    image_size = image_size,
    batch_size = batch_size

)

Found 24991 files belonging to 2 classes.
Using 19993 files for training.


In [10]:
validation_dr = tf.keras.utils.image_dataset_from_directory(
    dataset_url,
    subset="validation",
    seed=123,
    validation_split=0.2,
    image_size = image_size,
    batch_size = batch_size

)

Found 24991 files belonging to 2 classes.
Using 4998 files for validation.


In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(256,256,3)),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=128, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(units=1, activation="sigmoid")

    ])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"] )
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_zoom (RandomZoom)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 256, 256, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(
    train_dr,
    epochs = 10,
    validation_data= validation_dr,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, mode="max"), tf.keras.callbacks.ModelCheckpoint("best_model_version.keras", save_best_only=True,monitor="val_accuracy", verbose=1)],
    verbose=1
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5308 - loss: 0.7063
Epoch 1: val_accuracy improved from -inf to 0.65366, saving model to best_model_version.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 871s 1s/step - accuracy: 0.5309 - loss: 0.7062 - val_accuracy: 0.6537 - val_loss: 0.6185
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6643 - loss: 0.6228
Epoch 2: val_accuracy improved from 0.65366 to 0.70148, saving model to best_model_version.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 841s 1s/step - accuracy: 0.6643 - loss: 0.6228 - val_accuracy: 0.7015 - val_loss: 0.5903
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7026 - loss: 0.5779
Epoch 3: val_accuracy improved from 0.70148 to 0.75570, saving model to best_model_version.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 830s 1s/step - accuracy: 0.7026 - loss: 0.5779 - val_accuracy: 0.7557 - val_loss: 0.5124
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7445 - loss: 0.5270
Epoch 4: val_accur